In [1]:
# Colab에서 NSMC 데이터를 다운로드하기 위한 셸 명령어
!git clone https://github.com/e9t/nsmc.git

# 필요한 패키지 설치
!pip install konlpy

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [2]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re
from collections import defaultdict # defaultdict는 키가 없을 경우 자동으로 기본값을 생성
from konlpy.tag import Okt

# NSMC 데이터 로드
train_data = pd.read_table('nsmc/ratings_train.txt')
test_data = pd.read_table('nsmc/ratings_test.txt')

train_data.shape, test_data.shape

train_data = train_data.iloc[:10000, :]
test_data = test_data.iloc[:1000, :]


In [5]:
train_data.iloc[:10, :]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화별반개도 아까움,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데 꽤 볼만한데 헐리우드식 화려함에만 너무 길들여져 있나,1


In [3]:

# 데이터 전처리 (특수문자 제거, 결측값 제거)
def clean_text(text):
    text = re.sub(r'[^가-힣0-9\s]', '', text)  # 한글, 숫자, 공백을 제외한 모든 문자 제거
    return text

train_data['document'] = train_data['document'].apply(lambda x: clean_text(str(x)))
test_data['document'] = test_data['document'].apply(lambda x: clean_text(str(x)))

# 결측값 제거
train_data = train_data.dropna(subset=['document'])
test_data = test_data.dropna(subset=['document'])

# N-gram 생성을 위한 함수
def generate_ngram(text, n):
    words = text.split()  # 공백을 기준으로 단어 분리
    ngrams = []
    for i in range(len(words) - n + 1):
        ngram = ' '.join(words[i:i + n])
        ngrams.append(ngram)
    return ngrams

# N-gram을 딕셔너리에 저장하는 함수
def build_ngram_dict(corpus, n):
    ngram_dict = defaultdict(int)
    for sentence in corpus:
        ngrams = generate_ngram(sentence, n)
        for ngram in ngrams:
            ngram_dict[ngram] += 1
    return ngram_dict

# N-gram 벡터화 함수 (문장을 N-gram 벡터로 변환)
def ngram_vectorize(corpus, ngram_dict):
    vectors = []
    for sentence in corpus:
        ngram_vector = [0] * len(ngram_dict)  # N-gram 수만큼 0으로 채운 리스트
        ngrams = generate_ngram(sentence, 2)  # 2-gram 생성
        for ngram in ngrams:
            if ngram in ngram_dict:
                idx = list(ngram_dict.keys()).index(ngram)  # N-gram의 인덱스를 찾음
                ngram_vector[idx] += 1
        vectors.append(ngram_vector)
    return vectors

# N-gram 사전 구축 (1-gram + 2-gram 사용)
ngram_dict = build_ngram_dict(train_data['document'], 2)

# 훈련 데이터와 테스트 데이터를 N-gram 벡터로 변환
X_train = ngram_vectorize(train_data['document'], ngram_dict)
X_test = ngram_vectorize(test_data['document'], ngram_dict)

# 라벨 (긍정 = 1, 부정 = 0)
y_train = train_data['label']
y_test = test_data['label']

# 로지스틱 회귀 모델 학습
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 모델 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.5900

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.91      0.69       492
           1       0.76      0.28      0.41       508

    accuracy                           0.59      1000
   macro avg       0.65      0.59      0.55      1000
weighted avg       0.66      0.59      0.55      1000

